**(注意) 本ノートブックをtoorPIA分析パッケージ環境で実行しないでください。**


# toorPIA APIテスト用ノートブック

本ノートブックはGoogle Colabで作成しました。

現在のフォルダーに次のサンプルデータがあることを前提として動作します。
- `biopsy.csv`
- `biopsy-add.csv`

これらのファイルは、toorPIA分析パッケージ環境の次のファルダーにありまうので、これを適宜コピーしてご利用ください。

- `${HOME}/samples/csv-basic/rawdata`



## toorPIA APIライブラリのインストール

pipを利用して次のセルのように実行して手元の環境にインストールしてください。
なお、Google Colab以外の環境でも、pipが利用できる環境であれば同様にしてインストールすることができます。

In [ ]:
!pip install git+https://github.com/toorpia/toorpia.git

In [ ]:
import os

os.environ['TOORPIA_API_KEY'] = 'Valid_API_KEY'

## toorPIAライブラリの読み込み

クライアントプログラムからtoorPIAライブラリを読み込み、toorPIA()メソッドでクライアントインスタンスを生成します。

このとき、事前に環境変数`TOORPIA_API_KEY`にAPIキーを設定しておくか、`toorPIA()`への引数`api_key`を用いてAPIキーを設定します。

ここで正しいAPIキーが設定されていないとtoorPIA APIを利用することはできません。

In [ ]:
from toorpia import toorPIA
tp = toorPIA()
# tp = toorPIA(api_key="Valid API Key")

## toorPIA APIによる解析例

ここでは、`biopsy.csv`を用いてtoorPIAのベースマップを作成し、そのあと、このベースマップに対して`biopsy-add.csv`を追加プロットする方法を例示します。

### `biopsy.csv`の読み込み

toorPIA APIは、pandasのDataFrame形式のデータを解析することができます。
したがって、与えられたCSVファイルをpandasライブラリで読み込んでDataFrame形式にします。
このとき、解析に用いない（重み０とする）項目についてはdropしておく必要があります。

In [ ]:
import pandas as pd
df = pd.read_csv('biopsy.csv')
df = df.drop(columns=["No", "ID", "Diagnosis"])
df

### `fit_transform()`関数によるベースマップ作成

次のセルのように実行することでベースマップを作成することができます。
返り値はNumPy Array形式で、ここにベースマップの座標値が含まれます。

In [ ]:
res = tp.fit_transform(df)
res

### ベースマップのプロット

`matplotlib`を用いてベースマップのプロットを作成してみます。

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(res[:,0], res[:,1])
plt.show()

### 追加プロットの作成

追加プロットデータ`biopsy-add.csv`を、さきほど生成したベースマップに対してプロットしてみます。

データの読み込みはベースマップの時と同様にしてください。

In [ ]:
df_add = pd.read_csv('biopsy-add.csv')
df_add = df_add.drop(columns=["No", "ID", "Diagnosis"])
df_add

次のセルにあるとおり`addplot()`メソッドで追加プロットを作成することができます。

In [ ]:
res_add = tp.addplot(df_add)
res_add

In [ ]:
plt.scatter(res[:,0], res[:,1])
plt.scatter(res_add[:,0], res_add[:,1])
plt.show()

## 補足：ベースマップのID番号の取得

`tp.fit_transform()`関数でベースマップを作成した直後に、そのベースマップのID番号を取得しておくことで、あとで、特定のベースマップに対して追加プロットをすることができます。

In [ ]:
# ベースマップの作成と、ベースマップID番号の取得
res = tp.fit_transform(df)
print("toorPIA Map ID No: ", tp.mapNo)

# ベースマップID番号を指定して追加プロット
add_res = tp.addplot(df_add, tp.mapNo)

In [ ]:
plt.scatter(res[:,0], res[:,1])
plt.scatter(res_add[:,0], res_add[:,1])
plt.show()